In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
chat = ChatOpenAI(model="gpt-3.5-turbo",temperature=0.1)

prompt = template.format(country_a="Korea",country_b="Japan")

chat.predict(prompt)


'The distance between Korea and Japan varies depending on the specific locations being measured. On average, the distance between Seoul, South Korea and Tokyo, Japan is approximately 1,000 kilometers (620 miles) when measured in a straight line. However, the actual distance can be longer when taking into account the specific points of departure and arrival, as well as the mode of transportation being used.'

In [13]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# messages = [
#     SystemMessage(content="You are a geography expert. And you only reply in {language}."),
#     AIMessage(content="Ciao, mi chiamo {name}!"),
#     HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, What is your name?")
# ]

# chat.predict_messages(messages=messages)

template = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert. And you only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}!"),
    ("human","What is the distance between {country_a} and {country_b}. Also, What is your name?")
])

prompt = template.format_messages(language = "Japan",name = "Nakamura",country_a="Japan",country_b="Korea")

chat.predict_messages(prompt)

AIMessage(content='日本と韓国の距離は約900キロです。私の名前は中村です。')

In [23]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

p = CommaOutputParser()

p.parse('hello, how, are, you')



['hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."),
    ("human","{question}")
    
])

# prompt = template.format_messages(
#     max_items=10,
#     question="What are the colors?"
# )

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)



['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [27]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question":"What are best the K-League football Teams?"
})

['Ulsan Hyundai FC',
 'Jeonbuk Hyundai Motors',
 'Suwon Samsung Bluewings',
 'FC Seoul',
 'Pohang Steelers']

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model="gpt-3.5-turbo",temperature=0.1,streaming=True,callbacks=[StreamingStdOutCallbackHandler(),])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international football player. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food")
])

chef_chain = chef_prompt | chat



In [36]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegerarian. You find ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human","{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({
    "cuisine":"korean"
})


Sure! Here's a simple recipe for Bibimbap, a popular Korean dish:

Ingredients:
- 2 cups cooked white rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup spinach
- 1 cup bean sprouts
- 4 eggs
- 4 tbsp soy sauce
- 2 tbsp sesame oil
- 2 cloves garlic, minced
- 2 tbsp gochujang (Korean chili paste)
- Sesame seeds for garnish
- Vegetable oil for cooking

Instructions:
1. Cook the white rice according to package instructions and set aside.
2. In a small bowl, mix together soy sauce, sesame oil, minced garlic, and gochujang to make the sauce. Set aside.
3. Heat a bit of vegetable oil in a pan over medium heat. Add the julienned carrots and zucchini, and stir-fry for a few minutes until slightly softened. Remove from the pan and set aside.
4. In the same pan, add the spinach and bean sprouts. Stir-fry until wilted. Remove from the pan and set aside.
5. In the same pan, fry the eggs sunny side up or however you prefer.
6. To assemble the Bibimbap, divide the cooked rice into four bowls.

AIMessageChunk(content="For the vegetarian version of Bibimbap, you can follow the same recipe with a few adjustments:\n\n- Instead of using 4 eggs, you can substitute with tofu slices or a tofu scramble. Simply cut the tofu into slices or crumble it up and season it with a bit of soy sauce and sesame oil before cooking it in the pan.\n\n- If you prefer a vegan version, you can omit the eggs and use a vegan egg substitute or simply add more vegetables like mushrooms or bell peppers for added texture and flavor.\n\n- Make sure to use a vegetarian-friendly soy sauce that doesn't contain any fish sauce or other animal products.\n\n- You can also add some sliced shiitake mushrooms or marinated tofu for extra protein and umami flavor.\n\n- If you want to add some crunch to your Bibimbap, you can sprinkle some crushed roasted peanuts or sesame seeds on top before serving.\n\nEnjoy your vegetarian Bibimbap!")